# libraries

In [2]:
import pandas as pd
import json
import numpy as np
import math
import heapq

In [166]:
def compose_coordinate(latitude, longitude):
    composed_coordinate = '['+ str(longitude)+ ','+ str(latitude)+ ']'
    return composed_coordinate

# def haversine_distance(coord1, coord2):
#     R = 6371.0  # Earth's radius in km
#     lat1 = math.radians(coord1[0])
#     lon1 = math.radians(coord1[1])
#     lat2 = math.radians(coord2[0])
#     lon2 = math.radians(coord2[1])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     distance = R * c
#     return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c

    #distance = ((lat1-lat2)**2+(lon1-lon2)**2)**0.5



    return distance


# import data

In [4]:
total_node = pd.read_csv("./total_node.csv")
total_link = pd.read_csv("./total_link.csv")
print("노드의 크기:",total_node.shape)
print("링크의 크기:",total_link.shape)

노드의 크기: (461110, 2)
링크의 크기: (361526, 17)


### 노드 반올림

In [5]:
total_node['경도반올림'] = total_node['경도'].round(9)
total_node['위도반올림'] = total_node['위도'].round(9)

### 중복 노드 제거 및 링크 넘버링

In [6]:
total_node.drop_duplicates(keep='first')

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127
...,...,...,...,...
461102,127.001000,37.230355,127.001000,37.230355
461103,127.001046,37.230343,127.001046,37.230343
461107,127.025045,37.235468,127.025045,37.235468
461108,127.025000,37.235515,127.025000,37.235515


In [7]:
total_node.duplicated(['경도반올림','위도반올림']).sum()

108609

In [8]:
total_node.duplicated(['경도','위도']).sum()

108609

In [9]:
total_node.head()

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127


In [10]:
total_link.head()

,노드1,노드2,거리,경찰서,공원,cctv,가로등,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,"[126.8962667073345, 37.25877484423394]","[126.89626899673534, 37.25879828123242]",0.002614,9.941103,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[126.99492119734404, 37.25307704311938]","[126.99490426643689, 37.25311243627993]",0.004211,6.405129,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[127.00271514375935, 37.25412660967416]","[127.0023185009589, 37.254061429039]",0.035846,6.395166,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[127.0023185009589, 37.254061429039]","[127.00192790991031, 37.25400274184614]",0.035180,6.395011,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[127.00192790991031, 37.25400274184614]","[127.00142049616967, 37.25393654954797]",0.045509,6.394178,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#link의 weight을 만들었다. 나중에 거리와 위험도를 종합적으로 따져 weight값을 정할 것이고 현재는 그냥 거리값만 적용시켰다.
total_link.insert(2, 'weight', total_link['거리'].copy())



### 합쳐진 경도 위도 나누기

In [12]:
templink = pd.DataFrame( columns=['노드1경도', '노드1위도','노드2경도','노드2위도'])
for i in range(total_link.shape[0]):
    nodedata = json.loads(total_link['노드1'][i])
    nodedata.extend(json.loads(total_link['노드2'][i]))
    templink.loc[i] = nodedata

templink.head()


,노드1경도,노드1위도,노드2경도,노드2위도
0,126.896267,37.258775,126.896269,37.258798
1,126.994921,37.253077,126.994904,37.253112
2,127.002715,37.254127,127.002319,37.254061
3,127.002319,37.254061,127.001928,37.254003
4,127.001928,37.254003,127.001420,37.253937


### 반올림

In [13]:
#optional

templink['노드1경도'] = templink['노드1경도'].round(9)
templink['노드1위도'] = templink['노드1위도'].round(9)
templink['노드2경도'] = templink['노드2경도'].round(9)
templink['노드2위도'] = templink['노드2위도'].round(9)

### 노드 인덱스 연결

In [14]:
#optional

node1connect = []
node2connect = []

for i in range(templink.shape[0]):
    
    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드1위도'][i]) & (total_node['경도반올림'] == templink['노드1경도'][i])]
    try:
        node1connect.append(contemp.index[0])
    except:
        print("{}번째 데이터 [{},{}] not in node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))
        node1connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))

    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드2위도'][i]) & (total_node['경도반올림'] == templink['노드2경도'][i])]
    try:
        node2connect.append(contemp.index[0])
    except:
        print("{}번째 데이터[{},{}] not in node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))
        node2connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))

templink.insert(0, 'idx노드1', node1connect)
templink.insert(1, 'idx노드2', node2connect)


80273번째 데이터[37.326324665,126.957343228] not in node
80274번째 데이터 [37.326324665,126.957343228] not in node
133407번째 데이터[37.218289837,126.964691178] not in node
133408번째 데이터 [37.218289837,126.964691178] not in node


## 임시로 위 2개 데이터 삭제할것

In [15]:
total_link = pd.concat([templink, total_link], axis=1)

total_link = total_link.drop([80273, 80274, 133407, 133408])


total_link.head()


,idx노드1,idx노드2,노드1경도,노드1위도,노드2경도,노드2위도,노드1,노드2,weight,거리,...,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,0.0,1.0,126.896267,37.258775,126.896269,37.258798,"[126.8962667073345, 37.25877484423394]","[126.89626899673534, 37.25879828123242]",0.002614,0.002614,...,0,0,0,0,0,0,0,0,0,0
1,2.0,3.0,126.994921,37.253077,126.994904,37.253112,"[126.99492119734404, 37.25307704311938]","[126.99490426643689, 37.25311243627993]",0.004211,0.004211,...,0,0,0,0,0,0,0,0,0,0
2,4.0,5.0,127.002715,37.254127,127.002319,37.254061,"[127.00271514375935, 37.25412660967416]","[127.0023185009589, 37.254061429039]",0.035846,0.035846,...,0,0,0,0,0,0,0,0,0,0
3,5.0,6.0,127.002319,37.254061,127.001928,37.254003,"[127.0023185009589, 37.254061429039]","[127.00192790991031, 37.25400274184614]",0.035180,0.035180,...,0,0,0,0,0,0,0,0,0,0
4,6.0,7.0,127.001928,37.254003,127.001420,37.253937,"[127.00192790991031, 37.25400274184614]","[127.00142049616967, 37.25393654954797]",0.045509,0.045509,...,0,0,0,0,0,0,0,0,0,0


In [16]:
total_link.to_csv("D:/modified_link.csv", index=True)

# 가중치 재설정

In [ ]:
weights = pd.read_csv("C:/Users/gimm7/OneDrive/바탕 화면/prediction_link.csv")
weights_re = weights.loc[:,['거리', '경찰서', 'cctv', '가로등', '보안등', '위험도']]
weights.drop(['weight', '거리', '경찰서', 'cctv', '가로등', '보안등', '위험도'], axis=1, inplace=True)
colunms = ['거리', '경찰서', 'cctv', '가로등', '보안등', '위험도']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
weights_re = scaler.fit_transform(weights_re)
weights_re = pd.DataFrame(weights_re, columns=colunms)

print(weights_re.head())
# 작을수록 좋은 거 -> 거리, 경찰서, 위험도
# 클수록 좋은 거 -> cctv, 가로등, 보안등
# 다익스트라는 가중치를 최소화하는 길을 찾는 것 
for i in range(len(weights_re)):
    a = weights_re.loc[i,['cctv']]
    weights_re.loc[i,['cctv']] = (-1)*a
    b = weights_re.loc[i,['가로등']]
    weights_re.loc[i,['가로등']] = (-1)*b
    c = weights_re.loc[i,['보안등']]
    weights_re.loc[i,['보안등']] = (-1)*c
print(weights_re.head())

In [ ]:
last_weights = weights_re.sum(axis=1)
last_weights = pd.DataFrame(last_weights, columns=["last_weight"])
last_weights['last_weight'] = (last_weights['last_weight'] + 100)/ 10000
total = pd.concat([weights, last_weights], axis=1)

In [ ]:
total.to_csv("C:/Users/gimm7/OneDrive/바탕 화면/last_weight_arranegment.csv")